# SuperstoreSalesPredictor

This Jupyter Notebook analyzes Superstore sales data made available on [Kaggle](https://www.kaggle.com/jr2ngb/superstore-data).  This machine learning sales predictor uses time series analysis to precict a year of sales.  It was created for INFO 659 at Drexel University for the Fall 2021-2022 Quarter.  This `.ipynb` file -- and all relevant documentation -- is available on [Github](https://github.com/zachcarlson/SuperstoreSalesPredictor)

The `SuperstoreSalesPredictor` project was coded in both Python and R.

## Group Information

### Team Members:

- Zach Carlson, zc378@drexel.edu
- Sarah Haley, slh54@drexel.edu
- Nancy Melucci, njm99@drexel.edu

## Importing Data and Preprocessing
### Import Packages

In [2]:
import pandas as pd
pd.set_option('max_columns', 25) #some columns will be truncated otherwise, setting to number of columns in sales

In [3]:
def daily_data(data):
    daily_data = data.copy()
    daily_data.date = daily_data.date.apply(lambda x: str(x)[:-3])
    daily_data = daily_data.groupby('date')['sales'].sum().reset_index()
    daily_data.date = pd.to_datetime(monthly_data.date)
    return monthly_data

### Import Data and inspect dataset

In [4]:
sales_raw = pd.read_csv("./data/superstore_dataset2011-2015.csv", 
                    encoding = "latin")

In [5]:
sales_raw.head()

Row ID         Order ID Order Date Ship Date       Ship Mode Customer ID  \
0   42433     AG-2011-2040   1/1/2011  6/1/2011  Standard Class    TB-11280   
1   22253    IN-2011-47883   1/1/2011  8/1/2011  Standard Class    JH-15985   
2   48883     HU-2011-1220   1/1/2011  5/1/2011    Second Class      AT-735   
3   11731  IT-2011-3647632   1/1/2011  5/1/2011    Second Class    EM-14140   
4   22255    IN-2011-47883   1/1/2011  8/1/2011  Standard Class    JH-15985   

     Customer Name      Segment         City            State    Country  \
0  Toby Braunhardt     Consumer  Constantine      Constantine    Algeria   
1      Joseph Holt     Consumer  Wagga Wagga  New South Wales  Australia   
2    Annie Thurman     Consumer     Budapest         Budapest    Hungary   
3     Eugene Moren  Home Office    Stockholm        Stockholm     Sweden   
4      Joseph Holt     Consumer  Wagga Wagga  New South Wales  Australia   

   Postal Code  Market   Region        Product ID         Category  \
0          NaN  Africa   Africa  OFF-TEN-10000025  Office Supplies   
1          NaN    APAC  Oceania   OFF-SU-10000618  Office Supplies   
2          NaN    EMEA     EMEA  OFF-TEN-10001585  Office Supplies   
3          NaN      EU    North   OFF-PA-10001492  Office Supplies   
4          NaN    APAC  Oceania   FUR-FU-10003447        Furniture   

  Sub-Category                 Product Name    Sales  Quantity  Discount  \
0      Storage          Tenex Lockers, Blue  408.300         2       0.0   
1     Supplies     Acme Trimmer, High Speed  120.366         3       0.1   
2      Storage      Tenex Box, Single Width   66.120         4       0.0   
3        Paper  Enermax Note Cards, Premium   44.865         3       0.5   
4  Furnishings   Eldon Light Bulb, Duo Pack  113.670         5       0.1   

    Profit  Shipping Cost Order Priority  
0  106.140          35.46         Medium  
1   36.036           9.72         Medium  
2   29.640           8.17           High  
3  -26.055           4.82           High  
4   37.770           4.70         Medium

In [6]:
sales_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Row ID          51290 non-null  int64  
 1   Order ID        51290 non-null  object 
 2   Order Date      51290 non-null  object 
 3   Ship Date       51290 non-null  object 
 4   Ship Mode       51290 non-null  object 
 5   Customer ID     51290 non-null  object 
 6   Customer Name   51290 non-null  object 
 7   Segment         51290 non-null  object 
 8   City            51290 non-null  object 
 9   State           51290 non-null  object 
 10  Country         51290 non-null  object 
 11  Postal Code     9994 non-null   float64
 12  Market          51290 non-null  object 
 13  Region          51290 non-null  object 
 14  Product ID      51290 non-null  object 
 15  Category        51290 non-null  object 
 16  Sub-Category    51290 non-null  object 
 17  Product Name    51290 non-null 

#### Observations:

- There are 51,290 instances with 24 features.
- Dates are in `DD/MM/YYYY` format.
- Features are mostly `object`, with five `float64` and two `int64` features.
- There are 41,296 missing values from `Postal Code`.  This is because the `Postal Code` is only present for U.S. purchases.

We'll save a copy of `sales_raw` in case we want to utilize `Postal Code` later.  However, for sales-focused prediction, it won't be necessary:

In [7]:
sales = sales_raw.copy()
sales = sales.drop("Postal Code", axis = 1) #axis=1 for column
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Row ID          51290 non-null  int64  
 1   Order ID        51290 non-null  object 
 2   Order Date      51290 non-null  object 
 3   Ship Date       51290 non-null  object 
 4   Ship Mode       51290 non-null  object 
 5   Customer ID     51290 non-null  object 
 6   Customer Name   51290 non-null  object 
 7   Segment         51290 non-null  object 
 8   City            51290 non-null  object 
 9   State           51290 non-null  object 
 10  Country         51290 non-null  object 
 11  Market          51290 non-null  object 
 12  Region          51290 non-null  object 
 13  Product ID      51290 non-null  object 
 14  Category        51290 non-null  object 
 15  Sub-Category    51290 non-null  object 
 16  Product Name    51290 non-null  object 
 17  Sales           51290 non-null 

## Time Series Analysis

In [9]:
#convert datetime columns to datetime objects
sales['Order Date'] = pd.to_datetime(sales['Order Date'])
sales['Ship Date'] = pd.to_datetime(sales['Ship Date'])

In [11]:
#Get only Order Date and Sales
sales = sales.loc[:, ["Order Date", "Sales"]]
sales 

Order Date    Sales
0     2011-01-01  408.300
1     2011-01-01  120.366
2     2011-01-01   66.120
3     2011-01-01   44.865
4     2011-01-01  113.670
...          ...      ...
51285 2014-12-31   13.904
51286 2014-12-31    3.990
51287 2014-12-31   26.400
51288 2014-12-31    7.120
51289 2014-12-31    3.024

[51290 rows x 2 columns]

In [12]:
sales = sales.groupby("Order Date").sum()
sales

Sales
Order Date             
2011-01-01    808.56300
2011-01-02   2455.67000
2011-01-03  12565.76630
2011-01-04   2582.80100
2011-01-06  11480.49270
...                 ...
2014-12-27  13421.26376
2014-12-28   1647.17400
2014-12-29  25109.28878
2014-12-30  16831.61480
2014-12-31  13257.95430

[1430 rows x 1 columns]

## Conclusions and Summary